# Converting to py extension

In [ ]:
! jupyter nbconvert --to script Celsius-Fahrenheit.ipynb

# Creating Connection with AzureML

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import (
    Workspace,
    Datastore,
    Dataset,
    ScriptRunConfig,
    Environment,
    Experiment,
    Run,
)


auth = InteractiveLoginAuthentication(tenant_id="864324a0-fbca-44b4-a025-ee7be7426f0b")
ws = Workspace.from_config(auth=auth)

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Define the Conda dependencies
conda_dep = CondaDependencies().create(conda_packages=["tensorflow==2.10"])

# Create an Environment object
myenv = Environment(name="myenv")
myenv.python.conda_dependencies = conda_dep

# Prepare Training Script

## Importing Libraries

In [ ]:
import numpy as np
import os

# import pickle
import tensorflow as tf

In [ ]:
run = Run.get_context()

## Disable Tensorflow logging

In [ ]:
tf.get_logger().setLevel("ERROR")

## Data Creation

In [ ]:
celsius_q = np.array([-40, -10, 0, 8, 15, 22, 38], dtype=float)
fahrenheit_a = np.array([-40, 14, 32, 46, 59, 72, 100], dtype=float)

## Model Creation

In [ ]:
l0 = tf.keras.layers.Dense(units=1, input_shape=[1])
model = tf.keras.Sequential([l0])
model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(0.1))

## Training

In [ ]:
history = model.fit(celsius_q, fahrenheit_a, epochs=1, verbose=0)

## Save model

In [ ]:
os.makedirs("outputs", exist_ok=True)
model_file = os.path.join("outputs", "model.pkl")

# with open(model_file, "wb") as f:
#   pickle.dump(model, f)

model.save("model.h5")  # Saves the model to a file named "model.h5"

run.complete()

# Submit the experiment

## Load AzureML Workspace

In [ ]:
ws = Workspace.from_config()

## Create AzureML Experiment

In [ ]:
experiment_name = "Exp_1"
experiment = Experiment(workspace=ws, name=experiment_name)

## Create ScriptRunConfig

In [ ]:
script_config = ScriptRunConfig(
    source_directory=".",
    script="Celsius-Fahrenheit.py",
    compute_target="OHMS-ML",
    environment=myenv,
)

## Submit the experiment

In [ ]:
run = experiment.submit(config=script_config)
run.wait_for_completion(show_output=True)